In [107]:
import numpy as np
import igl
import meshplot as mp

In [108]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [123]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting up  the constraints

In [202]:
# Grid for point searches
class SpatialIndex:
    def __init__(self, points, resolution):
        self.resolution = resolution
        self.grid = {}
        self.points = points
        self._build(points)

    # Return point's cell coordinates in the grid 
    def _grid_coords(self, point):
        return tuple((point // self.resolution).astype(int))

    # Assign each point to its corresponding cell
    def _build(self, points):
        for i, point in enumerate(points):
            key = self._grid_coords(point)
            if key not in self.grid:
                self.grid[key] = []
            self.grid[key].append(i)

    # Return the neighboring cells of a point
    # (its cell included)
    def _neighboring_cells(self, point, radius=1):
        base = self._grid_coords(point)
        offsets = range(-radius, radius + 1)
        for dx in offsets:
            for dy in offsets:
                for dz in offsets:
                    yield (base[0] + dx, base[1] + dy, base[2] + dz)

    # Return the closest point in the grid
    def find_closest_point(self, query_point):
        best_dist = float('inf')
        best_index = -1
        for key in self._neighboring_cells(query_point):
            # Could be a non existing cell, so we check
            if key in self.grid:
                for i in self.grid[key]:
                    dist = np.linalg.norm(self.points[i] - query_point)
                    if np.linalg.norm(self.points[i] - query_point) < best_dist:
                        best_dist = dist
                        best_index = i
        return best_index

    # Return all points within a given radius
    def closest_points(self, query_point, radius):
        closest_points = []
        for key in self._neighboring_cells(query_point, 2):
            if key in self.grid:
                for i in self.grid[key]:
                    if np.linalg.norm(self.points[i] - query_point) < radius:
                        closest_points.append(i)
        return closest_points

In [193]:
# Add here the code to generate the additional points and constraints

# Function to find the closest point to as point in a set of points
def find_closest_point(point, points):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argmin(distances)

# Function to build constraint equations
def build_constraints(points, normals, epsilon_factor=0.01):
    # Compute the bounding box diagonal
    bbox_min  = np.min(points, axis=0)
    bbox_max  = np.max(points, axis=0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    # Compute epsilon
    epsilon = epsilon_factor * bbox_diag

    # Spatial Index
    grid_resolution = 3
    grid = SpatialIndex(points, grid_resolution)

    p = np.zeros((3*len(points), 3))
    f = np.zeros(3*len(points))

    for i, pi in enumerate(points):
        # Add constraint f(pi) = 0
        p[3 * i] = pi
        f[3 * i] = 0.0
        # Add pi+N/pi+2N's constraints
        for j, sign in enumerate([1, -1]):
            epsilon_current = epsilon
            while True:
                # Compute pi+N/pi+2N
                pi_N = pi + (sign * epsilon_current * normals[i])

                # Find the closest point to pi+N/pi+2N
                # If the point is the same, break
                closest_index = grid.find_closest_point(pi_N)
                debug_1 = find_closest_point(pi_N, points)
                
                assert (debug_1 == closest_index)

                if np.array_equal(points[closest_index], pi):
                    break
                # Otherwise, update epsilon and repeat
                epsilon_current /= 2
            # Append new point and constraints
            p[(3 * i) + j + 1] = pi_N
            f[(3 * i) + j + 1] = sign * epsilon
    
    p = np.array(p)
    f = np.array(f)
    
    return p, f

In [194]:
# Generate the new points and constraints
p, f = build_constraints(pi, ni)

# Create color vector
colors = np.zeros((p.shape[0], 3))
colors[0::3] = [0, 0, 1]
colors[1::3] = [1, 0, 0]
colors[2::3] = [0, 1, 0]
# Visualize the new point cloud 
mp.plot(p, c=colors, shading={"point_size": 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# MLS function

In [112]:
# This is mockup code, generating the distance field from the surface of the unit sphere in analytic form
# The code is provided to show the desired visualization
# Use tet_grid to generate the grid appropriate to your data
# Generate function fx at all nodes of the grid by evaluating the MLS function

---

In [113]:
# Parameters
bbox_min = np.min(p, axis=0)
bbox_max = np.max(p, axis=0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

# Enlarge slightly the bounding box
bbox_min -= 0.05 * (bbox_max - bbox_min)
bbox_max += 0.05 * (bbox_max - bbox_min)

# Resolution
n = 40
resolution = (n, n, n)

In [114]:
# Generate grid n x n x n
x, T = tet_grid(resolution, bbox_min, bbox_max)

In [208]:
# Function that retrieves the indices of all points in points 
# that are at distance less than h from point.
def closest_points(point, points, h):
    distances = np.linalg.norm(points - point, axis=1)
    return np.argwhere(distances < h).flatten()

def wendland_weight(p, p_i, radius):
    # normalized distance
    r = np.linalg.norm(p_i - p, axis=1)
    q = r / radius
    return (1 - q) ** 4 * (4 * q + 1) * (q < 1)

# Return the polynomial basis functions
# depending on the degree required
def pol_basis(x, degree):
    if degree == 0:
        return np.array([1])
    elif degree == 1:
        return np.array([1, x[0], x[1], x[2]])
    elif degree == 2:
        return np.array([1, x[0], x[1], x[2], x[0]**2, x[1]**2, x[2]**2, 
                         x[0]*x[1], x[1]*x[2], x[2]*x[0]])
    else:
        raise ValueError("Degree must be 0, 1, or 2")

# Evaluate MLS for a given point
def evaluate_MLS(point, points, constraints, radius, k, grid=None):
    # Check if the grid is provided
    if grid is None:
        neighbors = closest_points(point, points, radius)
    else:
        neighbors_brute = closest_points(point, points, radius)
        neighbors = grid.closest_points(point, radius)
        assert np.array_equal(neighbors, neighbors_brute)
    
    #neighbors_brute = closest_points(point, points, radius)
    #assert np.array_equal(neighbors, neighbors_brute)
    # If the number of constraint points is less than twice 
    # the number of polynomial coefficients, return a large positive value
    if len(neighbors) < ((3/2) * k**2 + (3/2) * k + 1):
        return 10e6 
    
    # Linear system components:
    # B is the polynomial basis evaluated at the neighbors
    B = np.array([pol_basis(points[i], k) for i in neighbors])
    # W is the Wendland weight matrix
    W = np.diag(wendland_weight(point, points[neighbors], radius))

    A = B.T @ W @ B
    b = B.T @ W @ constraints[neighbors]
    
    # Solve the linear system
    coeffs = np.linalg.solve(A, b)
    return pol_basis(point, k) @ coeffs

def evaluate_MLS_on_grid(x, points, constraints, h, polyDegree):
    # Build the spatial index
    grid = SpatialIndex(points, 3)

    fx = np.array([evaluate_MLS(xi, points, constraints, h, polyDegree) for xi in x])
    return fx

In [209]:
# Parameters
wendlandRadius = 0.2 * bbox_diag
polyDegree = 2

fx = evaluate_MLS_on_grid(x, p, f, wendlandRadius, polyDegree)

In [210]:
# Assign colors
grid_colors = np.zeros((len(x), 3))
grid_colors[fx >= 0] = [1, 0, 0]  # Red = outside
grid_colors[fx <  0] = [0, 1, 0]  # Green = inside

# Visualize the grid
mp.plot(x, c=grid_colors, shading={"point_size": 6})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0848894…

---

# Marching to extract surface

In [211]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)

# Plot noisy surface
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0848894…

In [212]:
# Let's filter out smaller, noisy components

# Count number of faces per component
C = igl.face_components(sf)
component_sizes = np.bincount(C)

# Filter out components with less than min_faces
min_faces = 5000
sf_filtered = sf[component_sizes[C] >= min_faces]

# Keep only used vertices
used_vertices = np.unique(sf_filtered)
# Map from old to new vertex ids
reindex = -np.ones(sv.shape[0], dtype=int)
reindex[used_vertices] = np.arange(len(used_vertices))
# Remaining vertices and faces
sv_filtered = sv[used_vertices]
sf_filtered = reindex[sf_filtered]

# Plot the cleaned surface
mp.plot(sv_filtered, sf_filtered, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.3654203…